In [1]:
from models.DAX import DAX_quantile_regression, DAX_baseline
from models.energy import energy_models
from functions import get_energy,get_DAX,merge_submissions,check_submission
from functions.prepare_data import split_time
from functions.evaluation import evaluate_horizon
from tqdm import tqdm
from functions import evaluate_dax
import numpy as np

In [2]:
from functions import naive_ensemble

In [3]:
daxdata = get_DAX.get()

In [4]:
base = DAX_baseline.DAX_baseline(daxdata)

In [5]:
base

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-29,DAX,1 day,-3.130739,-0.517481,0.070684,0.718274,2.719772
1,2024-01-29,DAX,2 day,-4.060268,-0.781543,0.096673,0.987322,3.570492
2,2024-01-29,DAX,5 day,-4.877300,-0.880043,0.166567,1.255618,4.628697
3,2024-01-29,DAX,6 day,-5.766996,-0.944913,0.212652,1.451656,5.367396
4,2024-01-29,DAX,7 day,-6.206620,-1.023275,0.342612,1.594461,6.259934


In [6]:
df_dax=base

In [7]:
energydata= get_energy.get()

  0%|          | 0/267 [00:00<?, ?it/s]/Users/stephantimpe/PycharmProjects/probabilistic-forecasting-challenge/functions/get_energy.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 267/267 [00:29<00:00,  9.12it/s]


In [8]:


df_energy= energy_models.energy_XGBoost(energydata,date_str='2024-01-29')

In [9]:
df_energy

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-29,energy,36 hour,65.226463,73.813988,75.437737,73.815346,77.105888
1,2024-01-29,energy,40 hour,61.277119,70.505981,68.728668,70.618698,75.327736
2,2024-01-29,energy,44 hour,62.533669,67.075638,67.317947,68.031296,71.926605
3,2024-01-29,energy,60 hour,64.233055,71.542496,71.478424,71.273369,78.631477
4,2024-01-29,energy,64 hour,61.660507,68.383492,68.656525,69.429321,76.625282
5,2024-01-29,energy,68 hour,61.406391,65.026924,64.245865,66.678711,71.979507


In [10]:
#Reorder quantiles to prevent quantile crossing

In [11]:
def reorder_quantiles(df):
    quantile_cols = ['q0.025', 'q0.25', 'q0.5', 'q0.75', 'q0.975']
    # Sorting quantile values row-wise
    df_sorted = df.copy()
    for index, row in df.iterrows():
        sorted_quantiles = np.sort(row[quantile_cols])
        df_sorted.loc[index, quantile_cols] = sorted_quantiles
    return df_sorted

df_reordered = reorder_quantiles(df_energy)
df_reordered

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-29,energy,36 hour,65.226463,73.813988,73.815346,75.437737,77.105888
1,2024-01-29,energy,40 hour,61.277119,68.728668,70.505981,70.618698,75.327736
2,2024-01-29,energy,44 hour,62.533669,67.075638,67.317947,68.031296,71.926605
3,2024-01-29,energy,60 hour,64.233055,71.273369,71.478424,71.542496,78.631477
4,2024-01-29,energy,64 hour,61.660507,68.383492,68.656525,69.429321,76.625282
5,2024-01-29,energy,68 hour,61.406391,64.245865,65.026924,66.678711,71.979507


In [12]:
df = merge_submissions.merge(df_reordered,df_dax)

  forecast_date      target  horizon     q0.025      q0.25       q0.5  \
0    2024-01-29         DAX    1 day  -3.130739  -0.517481   0.070684   
1    2024-01-29         DAX    2 day  -4.060268  -0.781543   0.096673   
2    2024-01-29         DAX    5 day    -4.8773  -0.880043   0.166567   
3    2024-01-29         DAX    6 day  -5.766996  -0.944913   0.212652   
4    2024-01-29         DAX    7 day   -6.20662  -1.023275   0.342612   
0    2024-01-29      energy  36 hour  65.226463  73.813988  73.815346   
1    2024-01-29      energy  40 hour  61.277119  68.728668  70.505981   
2    2024-01-29      energy  44 hour  62.533669  67.075638  67.317947   
3    2024-01-29      energy  60 hour  64.233055  71.273369  71.478424   
4    2024-01-29      energy  64 hour  61.660507  68.383492  68.656525   
5    2024-01-29      energy  68 hour  61.406391  64.245865  65.026924   
0    2024-01-29  infections   0 week         NA         NA         NA   
1    2024-01-29  infections   1 week         NA    

In [13]:
import pandas as pd

In [14]:
# ensure forecast_date is in correct format
df['forecast_date'] = pd.to_datetime(df['forecast_date'], format='%Y-%m-%d')

In [15]:
df

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-29,DAX,1 day,-3.130739,-0.517481,0.070684,0.718274,2.719772
1,2024-01-29,DAX,2 day,-4.060268,-0.781543,0.096673,0.987322,3.570492
2,2024-01-29,DAX,5 day,-4.8773,-0.880043,0.166567,1.255618,4.628697
3,2024-01-29,DAX,6 day,-5.766996,-0.944913,0.212652,1.451656,5.367396
4,2024-01-29,DAX,7 day,-6.20662,-1.023275,0.342612,1.594461,6.259934
0,2024-01-29,energy,36 hour,65.226463,73.813988,73.815346,75.437737,77.105888
1,2024-01-29,energy,40 hour,61.277119,68.728668,70.505981,70.618698,75.327736
2,2024-01-29,energy,44 hour,62.533669,67.075638,67.317947,68.031296,71.926605
3,2024-01-29,energy,60 hour,64.233055,71.273369,71.478424,71.542496,78.631477
4,2024-01-29,energy,64 hour,61.660507,68.383492,68.656525,69.429321,76.625282


In [16]:
df.to_csv('../../forecasts/20240129_JonSnow.csv',index=False)

In [17]:
df

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-29,DAX,1 day,-3.130739,-0.517481,0.070684,0.718274,2.719772
1,2024-01-29,DAX,2 day,-4.060268,-0.781543,0.096673,0.987322,3.570492
2,2024-01-29,DAX,5 day,-4.8773,-0.880043,0.166567,1.255618,4.628697
3,2024-01-29,DAX,6 day,-5.766996,-0.944913,0.212652,1.451656,5.367396
4,2024-01-29,DAX,7 day,-6.20662,-1.023275,0.342612,1.594461,6.259934
0,2024-01-29,energy,36 hour,65.226463,73.813988,73.815346,75.437737,77.105888
1,2024-01-29,energy,40 hour,61.277119,68.728668,70.505981,70.618698,75.327736
2,2024-01-29,energy,44 hour,62.533669,67.075638,67.317947,68.031296,71.926605
3,2024-01-29,energy,60 hour,64.233055,71.273369,71.478424,71.542496,78.631477
4,2024-01-29,energy,64 hour,61.660507,68.383492,68.656525,69.429321,76.625282


In [18]:
from datetime import timedelta, datetime

In [19]:
from functions import check_submission

In [20]:
check_submission.check_df(df)

Start checking...
---------------------------
Checking the Columns...
Checking type of columns...
----WARNING: Some elements in q0.025 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

----WARNING: Some elements in q0.25 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

----WARNING: Some elements in q0.5 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

----WARNING: Some elements in q0.75 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

----WARNING: Some elements in q0.975 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

Checking if the Dates make sense...
----WARNING: Forecast date should be a Wednesday.

Checking targets...
Skipping plausibility check for 'infections' as it contains NA values.
Skipping plausibility check for 'infections' as it contains NA values.
Skipping plausibility check for 'infections' as it contains NA valu